In [1]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

devices = tf.config.list_physical_devices("GPU")
print(devices)
try:
    tf.config.experimental.set_memory_growth(devices[0], True)
except:
    pass


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Download the IMDB dataset

- `tensorflow-dataset`에서 'imdb reviews' 데이터세트를 다운로드

In [2]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

### Explore the data

In [3]:
train_example_batch, train_label_batch = next(iter(train_data.batch(3)))
train_example_batch

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [4]:
train_label_batch

<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 0, 0], dtype=int64)>

### Build the model

신경망은 레이어를 쌓음으로써 생성된다. 이 레이어를 쌓는 데에는 3 가지의 주요 아키텍처 결정이 필요하다.
- **텍스트를 표현하는 방법은?**
- **모델에 몇 층의 레이어를 쌓을 것인가?**
- **각 레이어에 사용할 은닉 유닛은 몇 개인가?**

이 연습에서는 입력데이터는 문장으로 구성된다. 또한 예측할 레이블은 0 또는 1 이다.

텍스트를 표현하는 한 가지 방법은 문장을 임베딩 벡터로 변환하는 것이다. 사전 훈련된 텍스트 임베딩을 첫 번째 레이어로 사용하면 3 가지 이점이 있다.
- 텍스트 전처리에 대해 걱정할 필요가 없다.
- 전이 학습의 혜택
- 임베딩은 크기가 고정되어 있어서 처리가 더 간단하다.

*더 큰 차원의 임베딩은 성능을 향상시킬 수 있지만, 모델을 학습시키는 데 더 오래 걸릴 수 있다.*<br>
*텍스트 정규화는 텍스트에 추가 문자나 구두점이 포함된 경우 도움이 될 수 있다.*<br>

**입력 텍스트의 길이에 관계없이 임베딩의 출력 모양은 `(num_examples, embedding_dimention)` 이다.**


In [6]:
embedding = 'https://tfhub.dev/google/nnlm-en-dim50/2'
hub_layer = hub.KerasLayer(
    embedding,
    input_shape=[],
    dtype=tf.string,
    trainable=True
    )
hub_layer(train_example_batch[:1])

<tf.Tensor: shape=(1, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456]],
      dtype=float32)>

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


### Loss function and optimizer

In [8]:
model.compile(
    optimizer='adam',
    loss=tf.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
    )

### Train the model

In [9]:
history = model.fit(
    train_data.shuffle(10000).batch(512),
    epochs=10,
    validation_data=validation_data.batch(512),
    verbose=1
)

Epoch 1/10
30/30 [==============================] - 6s 109ms/step - loss: 0.6595 - accuracy: 0.5311 - val_loss: 0.6009 - val_accuracy: 0.6076
Epoch 2/10
30/30 [==============================] - 3s 97ms/step - loss: 0.5259 - accuracy: 0.7024 - val_loss: 0.4799 - val_accuracy: 0.7601
Epoch 3/10
30/30 [==============================] - 3s 92ms/step - loss: 0.3860 - accuracy: 0.8299 - val_loss: 0.3864 - val_accuracy: 0.8273
Epoch 4/10
30/30 [==============================] - 3s 96ms/step - loss: 0.2791 - accuracy: 0.8936 - val_loss: 0.3421 - val_accuracy: 0.8564
Epoch 5/10
30/30 [==============================] - 3s 94ms/step - loss: 0.2043 - accuracy: 0.9301 - val_loss: 0.3177 - val_accuracy: 0.8643
Epoch 6/10
30/30 [==============================] - 3s 91ms/step - loss: 0.1490 - accuracy: 0.9546 - val_loss: 0.3086 - val_accuracy: 0.8639
Epoch 7/10
30/30 [==============================] - 3s 100ms/step - loss: 0.1081 - accuracy: 0.9699 - val_loss: 0.3107 - val_accuracy: 0.8706
Epoch 8/10


### Evaluate the model

In [10]:
result = model.evaluate(test_data.batch(512), verbose=2)

49/49 - 2s - loss: 0.3706 - accuracy: 0.8535 - 2s/epoch - 37ms/step


In [11]:
for name, value in zip(model.metrics_names, result):
    print("%s: %.3f" % (name, value))

loss: 0.371
accuracy: 0.854
